In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import multiprocessing as mp
from tqdm import tqdm


In [4]:
df_total = pd.read_csv('choice_data.csv',sep=',',index_col=0)
df_rabbit = pd.read_csv('rabbit_data.csv',sep=',',index_col=0)

sum_choice_seq = df_total.groupby('worker_id')['choice_value'].sum().to_frame()
analysis_workers = sum_choice_seq[(sum_choice_seq['choice_value'] != 24) & (sum_choice_seq['choice_value'] != 0)].index
df_analysis = df_total[df_total['worker_id'].isin(analysis_workers)]

In [6]:
n_iter = 10
pool_size = 8
estimates = []

def fit_bootstrap(sample_index, data):
    bootstrap_sample = data.sample(n=len(data), replace=True)
    mod_bootstrap = smf.logit('choice_value ~ group_value + part_value + I((group_value==1)*part_value) + C(single_amount) + C(front_ratio)', data=bootstrap_sample)
    result_bootstrap = mod_bootstrap.fit(disp=0)  
    return result_bootstrap.params
    
with mp.Pool(pool_size) as pool:
        args = [(i, df_analysis) for i in range(n_iter)]

        for result in tqdm(
                            pool.imap_unordered(fit_bootstrap, args),
                            total=n_iter):
            
            estimates += [result]

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
mp.cpu_count()

8

In [5]:
bootstrap_estimates_df = pd.DataFrame(estimates)

bootstrap_means = bootstrap_estimates_df.mean()
bootstrap_se = bootstrap_estimates_df.std()
bootstrap_ci_lower = bootstrap_estimates_df.quantile(0.025)
bootstrap_ci_upper = bootstrap_estimates_df.quantile(0.975)

# Print results
print("Bootstrap Means:")
print(bootstrap_means)
print("\nBootstrap Standard Errors:")
print(bootstrap_se)
print("\nBootstrap 95% Confidence Intervals:")
print(pd.DataFrame({'lower': bootstrap_ci_lower, 'upper': bootstrap_ci_upper}))

d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is de

Intercept                                    -0.236635
group_value[T.True]                           0.016963
part_value[T.True]                            0.075606
I((group_value == 1) * part_value)[T.True]    0.165231
C(single_amount)[T.240]                       0.423976
C(single_amount)[T.280]                       1.042799
C(single_amount)[T.320]                       0.890020
C(front_ratio)[T.0.2]                        -0.775973
C(front_ratio)[T.0.3]                        -0.688705
C(front_ratio)[T.0.4]                        -0.463887
C(front_ratio)[T.0.5]                        -0.467867
C(front_ratio)[T.0.6]                        -0.924494
dtype: float64

In [5]:
def fit_bootstrap(sample_index, data):
    bootstrap_sample = data.sample(n=len(data), replace=True)
    mod_bootstrap = smf.logit('choice_value ~ group_value + part_value + I((group_value==1)*part_value) + C(single_amount) + C(front_ratio)', data=bootstrap_sample)
    result_bootstrap = mod_bootstrap.fit(disp=0)  
    return result_bootstrap.params

In [11]:
ser1 = fit_bootstrap(1, df_analysis)
ser2 = fit_bootstrap(2, df_analysis)

d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
d:\Python\lib\site-packages\patsy\util.py:672: FutureWarning: is_categorical_dtype is de

In [15]:
[ser1, ser2]

,Intercept,group_value[T.True],part_value[T.True],I((group_value == 1) * part_value)[T.True],C(single_amount)[T.240],C(single_amount)[T.280],C(single_amount)[T.320],C(front_ratio)[T.0.2],C(front_ratio)[T.0.3],C(front_ratio)[T.0.4],C(front_ratio)[T.0.5],C(front_ratio)[T.0.6]
0,-0.430434,-0.016302,0.182994,0.155565,0.413467,1.188382,0.753420,-0.413150,-0.652457,-0.531543,-0.379973,-0.663521
1,-0.176682,0.037758,0.193642,0.065159,0.267205,0.968430,0.755872,-0.644973,-0.718497,-0.771942,-0.576714,-0.841197


In [14]:
ser1.T

Intercept                                    -0.430434
group_value[T.True]                          -0.016302
part_value[T.True]                            0.182994
I((group_value == 1) * part_value)[T.True]    0.155565
C(single_amount)[T.240]                       0.413467
C(single_amount)[T.280]                       1.188382
C(single_amount)[T.320]                       0.753420
C(front_ratio)[T.0.2]                        -0.413150
C(front_ratio)[T.0.3]                        -0.652457
C(front_ratio)[T.0.4]                        -0.531543
C(front_ratio)[T.0.5]                        -0.379973
C(front_ratio)[T.0.6]                        -0.663521
dtype: float64